In [ ]:
# ══════════════════════════════════════════════════════════════════
# Trendline Breakout Navigator (TBN) - Showcase
# ══════════════════════════════════════════════════════════════════
#
# Multi-timeframe trendline detection ported from LuxAlgo PineScript.
# Detects pivot highs/lows at three swing lengths, constructs trendlines
# on HH/LL trend reversals, and tracks trendline breakouts and wick
# interactions.  Composite trend sums all three timeframes (-3 to +3).
# ══════════════════════════════════════════════════════════════════

from pathlib import Path
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pyindicators import (
    trendline_breakout_navigator,
    trendline_breakout_navigator_signal,
    get_trendline_breakout_navigator_stats,
)

# ── Data ───────────────────────────────────────────────────────────

ticker = "BTC/EUR (1d)"
df = pd.read_csv(
    "../../resources/data/OHLCV_BTC-EUR_BITVAVO_1d_2022-02-18-00-00_2026-02-17-00-00.csv",
    parse_dates=["Datetime"],
)
print(f"Loaded {len(df)} rows  |  {df['Datetime'].iloc[0]}  ->  {df['Datetime'].iloc[-1]}")

# ── Compute indicator ──────────────────────────────────────────────

df = trendline_breakout_navigator(
    df, swing_long=60, swing_medium=30, swing_short=10
)
df = trendline_breakout_navigator_signal(df)
stats = get_trendline_breakout_navigator_stats(df)

# ── Colours (dark theme — professional charting look) ──────────────

BG_COLOR    = "#131722"
PAPER_COLOR = "#131722"
GRID_COLOR  = "rgba(255,255,255,0.06)"
TEXT_COLOR  = "#d1d4dc"
TITLE_COLOR = "#e0e3eb"

CANDLE_UP   = "#26a69a"
CANDLE_DOWN = "#ef5350"

# Trendline colours — bright and vivid for dark background
TL_BULL_LONG    = "#00e676"        # Bright green — solid thick
TL_BEAR_LONG    = "#ff5252"        # Bright red — solid thick
TL_BULL_MED     = "rgba(0,230,118,0.70)"   # Green — dashed
TL_BEAR_MED     = "rgba(255,82,82,0.70)"   # Red — dashed
TL_BULL_SHORT   = "rgba(0,230,118,0.45)"   # Green — dotted
TL_BEAR_SHORT   = "rgba(255,82,82,0.45)"   # Red — dotted

# Marker colours
HH_COLOR        = "#00e676"
LL_COLOR        = "#ff5252"
WICK_BULL_COLOR = "#69f0ae"
WICK_BEAR_COLOR = "#ff8a80"

# Shading
BULL_SHADE  = "rgba(0,230,118,{a})"
BEAR_SHADE  = "rgba(255,82,82,{a})"

# ── Build figure (3 rows) ───────────────────────────────────────────

fig = make_subplots(
    rows=3, cols=1,
    row_heights=[0.70, 0.15, 0.15],
    shared_xaxes=True,
    vertical_spacing=0.025,
    subplot_titles=[
        f"<b>{ticker} — Trendline Breakout Navigator</b>",
        "<b>Composite Trend</b>",
        "<b>Volume</b>",
    ],
)

# ── Row 1: Candlestick + Trendlines + Markers ─────────────────────

fig.add_trace(go.Candlestick(
    x=df["Datetime"], open=df["Open"], high=df["High"],
    low=df["Low"], close=df["Close"], name=ticker,
    increasing_line_color=CANDLE_UP, decreasing_line_color=CANDLE_DOWN,
    increasing_fillcolor=CANDLE_UP, decreasing_fillcolor=CANDLE_DOWN,
    showlegend=False,
), row=1, col=1)

# --- Trendline overlays ---

trend_long = df["tbn_trend_long"].values
trend_medium = df["tbn_trend_medium"].values
trend_short = df["tbn_trend_short"].values
val_long = df["tbn_value_long"].values
val_medium = df["tbn_value_medium"].values
val_short = df["tbn_value_short"].values
composite = df["tbn_composite_trend"].fillna(0).astype(int).values

# Long trendline (bull) — solid, thick
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_long == 1, val_long, np.nan),
    mode="lines",
    line=dict(color=TL_BULL_LONG, width=2.5),
    name="Long TL ↑",
    connectgaps=False,
), row=1, col=1)

# Long trendline (bear) — solid, thick
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_long == -1, val_long, np.nan),
    mode="lines",
    line=dict(color=TL_BEAR_LONG, width=2.5),
    name="Long TL ↓",
    connectgaps=False,
), row=1, col=1)

# Medium trendline (bull) — dashed
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_medium == 1, val_medium, np.nan),
    mode="lines",
    line=dict(color=TL_BULL_MED, width=1.8, dash="dash"),
    name="Med TL ↑",
    connectgaps=False,
), row=1, col=1)

# Medium trendline (bear) — dashed
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_medium == -1, val_medium, np.nan),
    mode="lines",
    line=dict(color=TL_BEAR_MED, width=1.8, dash="dash"),
    name="Med TL ↓",
    connectgaps=False,
), row=1, col=1)

# Short trendline (bull) — dotted, thin
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_short == 1, val_short, np.nan),
    mode="lines",
    line=dict(color=TL_BULL_SHORT, width=1.2, dash="dot"),
    name="Short TL ↑",
    connectgaps=False,
), row=1, col=1)

# Short trendline (bear) — dotted, thin
fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=np.where(trend_short == -1, val_short, np.nan),
    mode="lines",
    line=dict(color=TL_BEAR_SHORT, width=1.2, dash="dot"),
    name="Short TL ↓",
    connectgaps=False,
), row=1, col=1)

# --- Event markers ---

# HH markers (triangle-up below low)
hh_mask = df["tbn_hh"] == 1
if hh_mask.any():
    fig.add_trace(go.Scatter(
        x=df.loc[hh_mask, "Datetime"],
        y=df.loc[hh_mask, "Low"] * 0.993,
        mode="markers", name="HH",
        marker=dict(
            symbol="triangle-up", size=12,
            color=HH_COLOR,
            line=dict(width=1.2, color="rgba(255,255,255,0.7)"),
        ),
    ), row=1, col=1)

# LL markers (triangle-down above high)
ll_mask = df["tbn_ll"] == 1
if ll_mask.any():
    fig.add_trace(go.Scatter(
        x=df.loc[ll_mask, "Datetime"],
        y=df.loc[ll_mask, "High"] * 1.007,
        mode="markers", name="LL",
        marker=dict(
            symbol="triangle-down", size=12,
            color=LL_COLOR,
            line=dict(width=1.2, color="rgba(255,255,255,0.7)"),
        ),
    ), row=1, col=1)

# Wick Bull markers (diamond)
wb_mask = df["tbn_wick_bull"] == 1
if wb_mask.any():
    fig.add_trace(go.Scatter(
        x=df.loc[wb_mask, "Datetime"],
        y=df.loc[wb_mask, "Low"] * 0.990,
        mode="markers", name="Wick Bull",
        marker=dict(
            symbol="diamond", size=9,
            color=WICK_BULL_COLOR,
            line=dict(width=1, color="rgba(255,255,255,0.5)"),
        ),
    ), row=1, col=1)

# Wick Bear markers (diamond)
wbr_mask = df["tbn_wick_bear"] == 1
if wbr_mask.any():
    fig.add_trace(go.Scatter(
        x=df.loc[wbr_mask, "Datetime"],
        y=df.loc[wbr_mask, "High"] * 1.010,
        mode="markers", name="Wick Bear",
        marker=dict(
            symbol="diamond", size=9,
            color=WICK_BEAR_COLOR,
            line=dict(width=1, color="rgba(255,255,255,0.5)"),
        ),
    ), row=1, col=1)

# --- Background shading by composite trend ---

prev_sign = 0
seg_start = 0

for i in range(len(df)):
    cur_sign = 1 if composite[i] > 0 else (-1 if composite[i] < 0 else 0)

    if cur_sign != prev_sign or i == len(df) - 1:
        if prev_sign != 0 and i > seg_start:
            end_idx = i if i < len(df) - 1 else i
            fill = BULL_SHADE.format(a=0.07) if prev_sign == 1 \
                else BEAR_SHADE.format(a=0.07)
            fig.add_vrect(
                x0=df.loc[seg_start, "Datetime"],
                x1=df.loc[min(end_idx, len(df) - 1), "Datetime"],
                fillcolor=fill, layer="below", line_width=0,
                row=1, col=1,
            )
        seg_start = i
        prev_sign = cur_sign

# ── Row 2: Composite Trend (bar chart) ────────────────────────────

comp_colors = [
    CANDLE_UP if c > 0 else CANDLE_DOWN if c < 0 else "#363a45"
    for c in composite
]

fig.add_trace(go.Bar(
    x=df["Datetime"], y=composite,
    marker_color=comp_colors, showlegend=False,
    name="Composite Trend", opacity=0.85,
), row=2, col=1)

fig.add_hline(
    y=0, line_dash="solid", line_color="rgba(255,255,255,0.15)", line_width=0.8,
    row=2, col=1,
)

# ── Row 3: Volume bars ─────────────────────────────────────────────

vol_colors = [
    CANDLE_UP if c > 0 else CANDLE_DOWN if c < 0 else "#363a45"
    for c in composite
]

fig.add_trace(go.Bar(
    x=df["Datetime"], y=df["Volume"],
    marker_color=vol_colors, showlegend=False,
    name="Volume", opacity=0.60,
), row=3, col=1)

# ── Layout ─────────────────────────────────────────────────────────

fig.update_layout(
    height=1000,
    width=1400,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom", y=1.02,
        xanchor="right", x=1,
        font=dict(size=10, color=TEXT_COLOR),
        bgcolor="rgba(0,0,0,0)",
        borderwidth=0,
    ),
    plot_bgcolor=BG_COLOR,
    paper_bgcolor=PAPER_COLOR,
    font=dict(color=TEXT_COLOR, size=12, family="Consolas, monospace"),
    margin=dict(l=10, r=70, t=60, b=30),
    xaxis_rangeslider_visible=False,
)

# Subplot title styling
for ann in fig.layout.annotations:
    ann.font.color = TITLE_COLOR
    ann.font.size = 13

for row in range(1, 4):
    fig.update_xaxes(
        showgrid=True, gridcolor=GRID_COLOR,
        zeroline=False, row=row, col=1,
        color=TEXT_COLOR,
        showline=False,
    )
    fig.update_yaxes(
        showgrid=True, gridcolor=GRID_COLOR,
        zeroline=False, side="right", row=row, col=1,
        color=TEXT_COLOR,
        showline=False,
    )

fig.update_yaxes(title_text="Price", tickformat=",.0f", row=1, col=1)
fig.update_yaxes(
    title_text="Composite", range=[-3.5, 3.5], dtick=1, row=2, col=1,
)
fig.update_yaxes(title_text="Volume", row=3, col=1)

# ── Save outputs ───────────────────────────────────────────────────

fig.write_html("trendline_breakout_navigator.html", auto_open=False)

output_path = Path.cwd().parent.parent / "static" / "images" / "indicators"
output_path.mkdir(parents=True, exist_ok=True)
fig.write_image(str(output_path / "trendline_breakout_navigator.png"), scale=2)

docs_path = Path.cwd().parent.parent / "docs" / "static" / "img" / "indicators"
docs_path.mkdir(parents=True, exist_ok=True)
fig.write_image(str(docs_path / "trendline_breakout_navigator.png"), scale=2)

fig.show()

# ── Print statistics ───────────────────────────────────────────────

print("=" * 55)
print(f"  Trendline Breakout Navigator - Stats ({ticker})")
print("=" * 55)
print(f"  Bullish Bars (Long):     {stats['bullish_bars_long']}")
print(f"  Bearish Bars (Long):     {stats['bearish_bars_long']}")
print(f"  Bullish Bars (Medium):   {stats['bullish_bars_medium']}")
print(f"  Bearish Bars (Medium):   {stats['bearish_bars_medium']}")
print(f"  Bullish Bars (Short):    {stats['bullish_bars_short']}")
print(f"  Bearish Bars (Short):    {stats['bearish_bars_short']}")
print("-" * 55)
print(f"  Composite Bullish:       {stats['composite_bullish']}  ({stats['composite_bullish_pct']:.1f}%)")
print(f"  Composite Bearish:       {stats['composite_bearish']}  ({stats['composite_bearish_pct']:.1f}%)")
print(f"  Trend Changes:           {stats['trend_changes']}")
print("-" * 55)
print(f"  HH Detections:           {stats['hh_count']}")
print(f"  LL Detections:           {stats['ll_count']}")
print(f"  Wick Bull Breaks:        {stats['wick_bull_count']}")
print(f"  Wick Bear Breaks:        {stats['wick_bear_count']}")
print(f"  Active Trendline Bars:   {stats['active_trendline_bars']}")
print("=" * 55)